In [ ]:
from datetime import datetime, timedelta

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from metpy.plots import StationPlot
import metpy.calc as mpcalc
from metpy.units import units
import matplotlib.pyplot as plt
import numpy as np
from siphon.simplewebservice.iastate import IAStateUpperAir
import xarray as xr

## Station Information

A helper function for obtaining radiosonde station information (e.g., latitude/longitude) requried to plot data obtained from each station. Original code by github user sgdecker.

In [ ]:
import urllib.request
import numpy as np

def station_info(stid):
    r"""Provide information about weather stations.
    
    Parameters
    ----------
    stid: str or iterable object containing strs
         The ICAO or IATA code(s) for which station information is requested.
    with_units: bool
         Whether to include units for values that have them. Default True.

    Returns
    -------
    info: dict
         Information about the station(s) within a dictionary with these keys:
             'state': Two-character ID of the state/province where the station is located,
                       if applicable
             'name': The name of the station
             'lat': The latitude of the station [deg]
             'lon': The longitude of the station [deg]
             'elevation': The elevation of the station [m]
             'country': Two-character ID of the country where the station is located
    
    Modified code from Steven Decker, Rutgers University

    """
    # Provide a helper function for later usage
    def str2latlon(s):
        deg = float(s[:3])
        mn = float(s[-3:-1])
        if s[-1] == 'S' or s[-1] == 'W':
            deg = -deg
            mn = -mn
        res = deg + mn / 60.
        return res

    # Various constants describing the underlying data
    url = 'https://www.aviationweather.gov/docs/metar/stations.txt'
    #file = 'stations.txt'
    state_bnds = slice(0, 2)
    name_bnds = slice(3, 19)
    icao_bnds = slice(20, 24)
    iata_bnds = slice(26, 29)
    lat_bnds = slice(39, 45)
    lon_bnds = slice(47, 54)
    z_bnds = slice(55, 59)
    cntry_bnds = slice(81, 83)

    # Generalize to any number of IDs
    if isinstance(stid, str):
        stid = [stid]

    # Get the station dataset
    infile = urllib.request.urlopen(url)
    data = infile.readlines()
#     infile = open(file, 'rb')
#     data = infile.readlines()

    state = []
    name = []
    lat = []
    lon = []
    z = []
    cntry = []
    
    for s in stid:
        s = s.upper()
        for line_bytes in data:
            line = line_bytes.decode('UTF-8')
            icao = line[icao_bnds]
            iata = line[iata_bnds]
            if len(s) == 3 and s in iata or len(s) == 4 and s in icao:
                state.append(line[state_bnds].strip())
                name.append(line[name_bnds].strip())
                lat.append(str2latlon(line[lat_bnds]))
                lon.append(str2latlon(line[lon_bnds]))
                z.append(float(line[z_bnds]))
                cntry.append(line[cntry_bnds])
                
                break
        else:
             #raise ValueError('Station {} not found!'.format(s))
            state.append('NA')
            name.append('NA')
            lat.append(np.nan)
            lon.append(np.nan)
            z.append(np.nan)
            cntry.append('NA')

    res = {'state': np.array(state), 'name': np.array(name), 'lat': np.array(lat), 'lon': np.array(lon), 'elevation': np.array(z),
           'country': np.array(cntry), 'units': {'lat': 'deg', 'lon': 'deg', 'z': 'm'}}
    infile.close()
        
    return res

In [ ]:
from scipy.ndimage import gaussian_filter
import pyproj

geod = pyproj.Geod(ellps='sphere')

def near_neighbor(xi, yi, xk, yk, phi):
    grid = np.zeros((len(yi),len(xi)))
    for e in range(len(xi)):
        for f in range(len(yi)):
            dum_dist = 10000000.
            for g in range(len(xk)):
                #dist = (((xk[g]-xi[e])**2) + ((yk[g]-yi[f])**2))**(0.5)
                _, _, dist = geod.inv(xk[g], yk[g], xi[e], yi[f])
                if (dist <= dum_dist):
                    dum_dist = dist
                    dum_phi = phi[g]
                elif (dist == dum_dist):
                    dum_phi = (dum_phi + phi[g])/2.
            grid[f,e] = dum_phi
    return gaussian_filter(grid, sigma=2)

def barnes_guess(x_i, y_i, x_k, y_k, phi_0, roi):
    grid = np.zeros((len(y_i),len(x_i)))
    # Loop through the grid and update the initial grid with the observations
    for l in range(len(x_i)):
        for m in range(len(y_i)):
            sum_weight = 0.
            num_weight = 0.
            weight = 0.
            for p in range(len(phi_0)): # Loop for the observations
                # Determining the distance between the observation and a grid point
                #rad2 = ((x_k[p]-x_i[l])**2 + (y_k[p]-y_i[m])**2)**(1/2)
                _, _, rad2 = geod.inv(x_k[p], y_k[p], x_i[l], y_i[m])
                weight = np.exp(-(rad2**2)/(roi**2))
                sum_weight+=weight
                weight_phi = weight * phi_0[p] # 
                num_weight+=weight_phi
                #print(rad2, weight)
            grid[m,l] = num_weight/sum_weight
    return gaussian_filter(grid, sigma=2)

def cressman(x_i, y_i, x_k, y_k, phi_0, guess_field):
    # Estimate of grid at Obs points
    h = np.zeros(len(phi_0)).astype('float')

    # Assimilated Grid
    grid = np.zeros((len(y_i),len(x_i)))
    #print(grid.shape)

    # Begin the Cressman Scheme, doing three passes with Radius of Influence (roi)
    # of 80, 50, and 40 grid points
    for i in [1,2]:
        if (i == 1):
            roi = (1500*1000)**2.
        elif (i == 2):
            roi = (900*1000.)**2.
        else:
            roi = 5.**2.

            # Calculating the values of the grid at the observation points through simple linear interpolation
        for p in range(len(phi_0)):
            #print(y_k[p]==np.nan, x_k[p])
            if (np.int64(y_k[p]) in y_i.astype('int64')) & ((np.int64(x_k[p]) in x_i.astype('int64'))):
                a = np.where(y_i.astype('int64') == np.int64(y_k[p]))[0][0]
                a2 = np.abs(y_i[a] - (y_k[p]))
                b = np.where(x_i.astype('int64') == np.int64(x_k[p]))[0][0]
                b2 = np.abs(x_i[b] - (x_k[p]))
                #print(a-1, a2, b-1, b2)
                h[p] = (((guess_field[a,b]*(1-b2)+guess_field[a,b+1]*b2))*(1-a2) +
                        ((guess_field[a-1,b]*(1-b2)+guess_field[a-1,b+1]*b2))*(a2))
            else:
                h[p] = phi_0[p]

    # Loop through the grid and update the initial grid with the observations
        for l in range(len(x_i)):
            for m in range(len(y_i)):
                sum_weight = 0.
                num_weight = 0.
                weight     = 0.
                for p in range(len(phi_0)): # Loop for the observations
                    # Determining the distance between the observation and a grid point
                    #rad2 = ((x_k[p]-x_i[l])**2 + (y_k[p]-y_i[m])**2)
                    _, _, rad2 = geod.inv(x_k[p], y_k[p], x_i[l], y_i[m])
                    rad2 = rad2**2
                    #print(dist2)
                    if (rad2 <= roi):
                        weight = (roi - rad2)/(roi + rad2) # Setting the weight if within the roi
                    else:
                        weight = 0. # Setting the weight of the ob to zero if outside the roi
                    sum_weight+=weight
                    weight_phi = weight * (phi_0[p] - h[p]) # 
                    num_weight+=weight_phi
                if sum_weight > 0:
                    grid[m,l] = guess_field[m,l] + (num_weight/sum_weight) # Updating the grid
                else:
                    grid[m,l] = guess_field[m,l]



        guess_field = grid   # Saving the grid to use in the next iteration
    return grid

def barnes(x_i, y_i, x_k, y_k, phi_0, guess_field):
    # Estimate of grid at Obs points
    h = np.zeros(len(phi_0)).astype('float')

    # Assimilated Grid
    grid = np.zeros((len(y_i),len(x_i)))

    # Begin the Cressman Scheme, doing three passes with Radius of Influence (roi)
    # of 80, 50, and 40 grid points
    roi = 1000*1000
    for i in [1,2]:
        if (i == 1):
            gamma = 1
        elif (i == 2):
            gamma = 0.33

        # Calculating the values of the grid at the observation points through simple linear interpolation
        for p in range(len(phi_0)):
            #print(np.int(y_k[p]) in y_i.astype('int'))
            if (np.int64(y_k[p]) in y_i.astype('int64')) & ((np.int64(x_k[p]) in x_i.astype('int64'))):
                #print('got here')
                a = np.where(y_i.astype('int64') == np.int64(y_k[p]))[0][0]
                a2 = np.abs(y_i[a] - (y_k[p]))
                b = np.where(x_i.astype('int64') == np.int64(x_k[p]))[0][0]
                b2 = np.abs(x_i[b] - (x_k[p]))
                h[p] = (((guess_field[a,b]*(1-b2)+guess_field[a,b+1]*b2))*(1-a2) +
                        ((guess_field[a-1,b]*(1-b2)+guess_field[a-1,b+1]*b2))*(a2))
            else:
                h[p] = phi_0[p]
        #print(h)

        # Loop through the grid and update the initial grid with the observations
        for l in range(len(x_i)):
            for m in range(len(y_i)):
                sum_weight = 0.
                num_weight = 0.
                weight = 0.
                for p in range(len(phi_0)): # Loop for the observations
                    # Determining the distance between the observation and a grid point
                    #rad2 = ((x_k[p]-x_i[l])**2 + (y_k[p]-y_i[m])**2)**(1/2)
                    _, _, rad2 = geod.inv(x_k[p], y_k[p], x_i[l], y_i[m])
                    weight = np.exp(-1*(rad2**2)/(gamma*(roi**2)))
                    sum_weight+=weight
                    weight_phi = weight * (phi_0[p] - h[p]) # 
                    num_weight+=weight_phi
                #print(num_weight, sum_weight)
                if sum_weight > 0:
                    grid[m,l] = guess_field[m,l] + (num_weight/sum_weight) # Updating the grid
                else:
                    grid[m,l] = guess_field[m,l]



        guess_field = grid   # Saving the grid to use in the next iteration
    return grid

In [ ]:
# Set date to do analysis
date = datetime(2017, 12, 9, 12)

# Request data using Siphon request for data from Iowa State Archive
data = IAStateUpperAir.request_all_data(date)

# Set level
level = 500

# Create subset of all data for a given level
data_subset = data.pressure == level
df = data[data_subset]

# Get station lat/lon from look-up file; add to Dataframe
stn_info = station_info(list(df.station.values))
df.insert(10, 'latitude', stn_info['lat'])
df.insert(11, 'longitude', stn_info['lon'])

df = df.dropna()

In [ ]:
lon = 360 + df.longitude.values
lat = df.latitude.values
hght_500 = df.height.values

In [ ]:
# Set up geographic region
LLLon = -140.
LLLat = 19.
URLon = -50.
URLat = 70

#ds = xr.open_dataset(f'http://www.ncei.noaa.gov/thredds/dodsC/model-gfs-g4-anl-files-old/'
#                     f'{date:%Y%m}/{date:%Y%m%d}/gfsanl_4_{date:%Y%m%d}_{date:%H}00_000.grb2')

ds = xr.open_dataset('/archive/courses/nwp/nwp_notebooks/gfsanl_3_20171209_1200_000.nc').metpy.parse_cf()

gfs_hght_500 = ds.Geopotential_height.metpy.sel(time=date,
                                                #vertical=500 * units.hPa,
                                                lon=slice(360+LLLon, 360+URLon),
                                                lat=slice(URLat, LLLat)).squeeze()

In [ ]:
def plot(FG_type):
    lons = gfs_hght_500.lon.values
    #lons[lons > 180] = lons - 360
    lats = gfs_hght_500.lat.values
    clons, clats = np.meshgrid(lons, lats)
    
    if FG_type == 'gfs':
        # Get First Guess field from 6 hour forecast of GFS
        ds_first_guess = xr.open_dataset('/archive/courses/nwp/nwp_notebooksgfs_3_20171209_0000_012.nc')
        date_first_guess = date - timedelta(hours=12)
        #ds_first_guess = xr.open_dataset(f'http://www.ncei.noaa.gov/thredds/dodsC/model-gfs-g3-anl-files-old/'
        #                                 f'{date_first_guess:%Y%m}/{date_first_guess:%Y%m%d}/gfsanl_3_{date_first_guess:%Y%m%d}_'
        #                                 f'{date_first_guess:%H}00_006.grb').metpy.parse_cf()

        FG_hght_500 = ds_first_guess.Geopotential_height.metpy.sel(time=date,
                                                                   #vertical=500 * units.hPa,
                                                                   lon=slice(360+LLLon, 360+URLon),
                                                                   lat=slice(URLat, LLLat)).squeeze()

        FG = FG_hght_500.values
        #FG_vtimes = FG_hght_500.time.values.astype('datetime64[ms]').astype('O')
        FG_vtimes = date
        #FG_vtimes = num2date(FG_time[:],units=FG_time.units)
        #FG_hght_500 = data_gfs_first_guess.variables['Geopotential_height'][0,0,:,:]
        #FG = FG_hght_500[iULat:iLLat,iLLon:iULon]
        FG_title = 'GFS Fcst'
    elif FG_type == 'NN':
        FG = near_neighbor(lons,lats,lon,lat,hght_500)
        FG_title = 'Nearest Neighbor'
    # Set firstguess field for Cressman Scheme
    # Use the Nearest Neighbor approach to initially fill the grid
    elif FG_type == 'barnes':
        FG = barnes_guess(lons,lats,lon,lat,hght_500,5e5)
        FG_title = 'Barnes'
    
    # Generate Analyzed Fields with Cressman and Barnes
    cress_hght_500 = cressman(lons,lats,lon,lat,hght_500,FG)
    barnes_hght_500 = barnes(lons,lats,lon,lat,hght_500,FG)

    print("First Guess Field: {}".format(FG_type))
    print("First Guess Error: {:.2f}".format(np.average(gfs_hght_500.values-FG)))
    print("Average Cressman Analysis Error: {:.2f}".format(np.average(gfs_hght_500.values-cress_hght_500)))
    print("Average Barnes Analysis Error: {:.2f}".format(np.average(gfs_hght_500.values-barnes_hght_500)))

    plotcrs = ccrs.PlateCarree()

    fig = plt.figure(1,figsize=(20,15))
    ax = fig.add_subplot(211,projection=plotcrs,label='top')

    ax.set_extent([-130, -65, 20, 50])
    ax.coastlines('50m')

    ax.add_feature(cfeature.STATES.with_scale('50m'), edgecolor='black', linewidth=0.5)
    # Set up station plotting using only every third element from arrays for plotting
    stationplot = StationPlot(ax, lon, lat, transform=ccrs.PlateCarree(), fontsize=12, clip_on=True)
    stationplot.plot_parameter('C', hght_500)

    cs = ax.contour(clons,clats,cress_hght_500,np.arange(0,7000,60),colors='r',linestyles='dashed')
    plt.clabel(cs,fmt='%d')
    cs3 = ax.contour(clons,clats,gfs_hght_500,np.arange(0,7000,60),colors='k')
    plt.clabel(cs3,fmt='%d')

    plt.title('500-hPa Geopotential Heights (Actual Analysis - black; Cressman Analysis - red)',loc='left')
    plt.title(date, loc='right')

    ax2 = fig.add_subplot(212, projection=plotcrs,label='bottom')
    ax2.set_extent([-130, -65, 20, 50])
    ax2.coastlines('50m')
    ax2.add_feature(cfeature.STATES.with_scale('50m'), edgecolor='black', linewidth=0.5)
    # Set up station plotting using only every third element from arrays for plotting
    stationplot = StationPlot(ax2, lon, lat, transform=ccrs.PlateCarree(), fontsize=12, clip_on=True)
    stationplot.plot_parameter('C', hght_500)

    cs2 = ax2.contour(clons,clats,barnes_hght_500,np.arange(0,7000,60),colors='b',linestyles='dotted')
    plt.clabel(cs2,fmt='%d')
    cs4 = ax2.contour(clons,clats,gfs_hght_500,np.arange(0,7000,60),colors='k')
    plt.clabel(cs4,fmt='%d')
    
    plt.title('500-hPa Geopotential Heights (Actual Analysis - black; Barnes Analysis - blue)',loc='left')
    plt.title(date, loc='right')

    #plt.tight_layout()
    plt.show()

In [ ]:
from ipywidgets import interact, widgets

interact(plot, FG_type = widgets.Dropdown(options={'GFS Fcst':'gfs','Nearest Neighbor':'NN','Barnes':'barnes'},
                                          description='First Guess: '));